In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import openai
import statistics

In [ ]:
sbc2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [ ]:
df = pd.read_csv('/content/etlPipelineSantanderBootcamp.csv')

user_ids = df['UserID'].tolist()

print(user_ids)

[1, 2, 3, 4, 5]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sbc2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

In [ ]:
balances = [user["account"]["balance"] for user in users]

median_balance = statistics.median(balances)


In [ ]:
#cruzar mais de uma api

In [ ]:
#se o balance tiver mto baixo, gerar mensagem sobre possibilidade de emprestimo/credito
#se o balance tiver mto alto, gerar mensagem sobre possibilidade de investimentos
#definir mto e pouco a partir do ponto que é a mediana entre os valores de balance de todos

In [ ]:
openai_api_key = 'TODO'

In [ ]:
openai.api_key = openai_api_key

def generate_ai_news(user):

  if user["account"]["balance"] <= median_balance:

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0301",
      messages=[
        {
            "role": "system",
            "content": "Você é um especialista em markting bancário."
        },
        {
            "role": "user",
            "content": f"Crie uma mensagem para {user['name']} sobre a possibilidade de empréstimos bancários (máximo de 100 caracteres)"
        }
      ]
    )

  else:

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0301",
      messages=[
        {
            "role": "system",
            "content": "Você é um especialista em markting bancário."
        },
        {
            "role": "user",
            "content": f"Crie uma mensagem para {user['name']} sobre a possibilidade de investimentos bancários (máximo de 100 caracteres)"
        }
      ]
    )

  return completion.choices[0].message.content.strip('\"')

In [ ]:
for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

In [ ]:
def update_user(user):
  response = requests.put(f"{sbc2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Devweekerson updated? False!
User Pip updated? True!
